In [1]:
# === The basic libs
import numpy as np
np.random.seed(123)  # for reproducibility
import pandas as pd
from matplotlib import pyplot as plt
# === tf and keras libs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
import tensorflow.keras.utils as np_utils

In [2]:
# === Settings needed not to run out of GPU memory
print(tf.version.VERSION)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

2.0.0
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:08:00.0, compute capability: 7.5
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



In [3]:
import os
os.getenv('HOST_PIPE_DIR')

'/home/rishin/workspace/ffx-ai/dist/'

In [4]:
NUM_CLASSES = 328
INPUT_SHAPE= (224, 224) # resnt, vgg, inception, mobilenet with default keras app take this only
DIST_PATH = os.getenv('HOST_PIPE_DIR')

In [5]:
def make_df(path):
    files = os.listdir(path)
    labels = [f.split('-')[0] for f in files]
    return pd.DataFrame({'file': files, 'label': labels})    

In [6]:
df_all = make_df(DIST_PATH)
df_all.head()

,file,label
0,mon_m021-7.png,mon_m021
1,mon_m302-5.png,mon_m302
2,mon_m326-7.png,mon_m326
3,mon_m007-1.png,mon_m007
4,mon_m289-0.png,mon_m289


In [7]:
assert NUM_CLASSES == len(df_all.label.unique())

In [8]:
def train_test_split(df, test_size=0.2, train_size=None):
    # determine train test split ratio
    df_len = len(df)
    test_size = test_size/df_len if isinstance(test_size, int) else test_size
    train_size = 1 - test_size if train_size is None else train_size
    ratio = train_size/df_len if isinstance(train_size, int) else train_size
    # split df
    msk = np.random.rand(df_len) < ratio
    train = df[msk]
    test = df[~msk]
    return train, test

In [9]:
df_train, df_test = train_test_split(df_all, test_size=NUM_CLASSES*2)

In [10]:
# https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c

In [13]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255., 
    rotation_range=22,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=DIST_PATH,
    x_col="file",
    y_col="label",
    batch_size=32,
    shuffle=True,
    class_mode="categorical",
    target_size=INPUT_SHAPE)

Found 1976 validated image filenames belonging to 328 classes.


In [14]:
test_datagen= tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=DIST_PATH,
    x_col="file",
    y_col="label",
    batch_size=32,
    shuffle=False,
    target_size=INPUT_SHAPE)

Found 648 validated image filenames belonging to 292 classes.


In [15]:
model = tf.keras.applications.mobilenet_v2.MobileNetV2(weights=None, classes=NUM_CLASSES)

In [16]:
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])

In [17]:
STEP_SIZE_TRAIN = train_generator.n / train_generator.batch_size
STEP_SIZE_TEST = test_generator.n / test_generator.batch_size

In [18]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=40
)

Epoch 1/40
62/61 [==============================] - 39s 627ms/step - loss: 6.2495 - accuracy: 0.0046
Epoch 2/40
62/61 [==============================] - 28s 451ms/step - loss: 5.4327 - accuracy: 0.0091
Epoch 3/40
62/61 [==============================] - 28s 455ms/step - loss: 4.9568 - accuracy: 0.0268
Epoch 4/40
62/61 [==============================] - 28s 448ms/step - loss: 4.4984 - accuracy: 0.0415
Epoch 5/40
62/61 [==============================] - 28s 444ms/step - loss: 4.0698 - accuracy: 0.0673
Epoch 6/40
62/61 [==============================] - 28s 445ms/step - loss: 3.6702 - accuracy: 0.1149
Epoch 7/40
62/61 [==============================] - 28s 445ms/step - loss: 3.3172 - accuracy: 0.1473
Epoch 8/40
62/61 [==============================] - 28s 448ms/step - loss: 3.0061 - accuracy: 0.1746
Epoch 9/40
62/61 [==============================] - 27s 441ms/step - loss: 2.6338 - accuracy: 0.2515
Epoch 10/40
62/61 [==============================] - 28s 444ms/step - loss: 2.4058 - accura

In [22]:
test_generator.reset()
y_pred = model.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=1)

21/20 [===============================] - 4s 167ms/step


In [29]:
predicted_class_indices = np.argmax(y_pred, axis=1)

In [31]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [32]:
predictions

['mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',
 'mon_m176',